# **Image Processing - 1st Assignment** 

> ## Image Histogram and Non Uniform Quantization






### Muhammad Kanzoon Mekhfy Asyraq Rekza
### 19/442479/PA/19228

## **Import Libraries and Image**

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np

img = cv.imread('nagyung.jpeg')
imgbnw = cv.cvtColor(img, cv.COLOR_BGR2GRAY) # Grayscale

## **Show The Image**

In [ ]:
cv.imshow("Nagyung",img)
cv.waitKey(0)
# Interupt to stop the process

In [ ]:
cv.imshow("Nagyung",imgbnw)
cv.waitKey(0)
# Interupt to stop the process

## **Show Image Histogram**

In [ ]:
plt.figure(figsize=(10,5))
plt.title('RGB Histogram')
plt.hist(img[:,:,0].ravel(), bins=256, range=(0, 256), color='blue', alpha = 0.4)
plt.hist(img[:,:,1].ravel(), bins=256, range=(0, 256), color='green', alpha = 0.4)
plt.hist(img[:,:,2].ravel(), bins=256, range=(0, 256), color='red', alpha = 0.3)
plt.show()

In [ ]:
# Histogram for bnw image
plt.figure(figsize=(10,5))
plt.title('Grayscale Histogram')
plt.hist(imgbnw.ravel(), bins=256, range=(0, 256), color='black', alpha=0.4)
plt.show()

## **Non Uniform Quantization Process**

Technique : Get mean of list, set as devide point, continue until K part, every part value equals to mean of its own part

In [ ]:
# Make new sorted array of imgbnw 
imgsorted = imgbnw.copy()
imgsorted = imgsorted.ravel()
imgsorted.sort()

# DUMMY DATA
# imgsorted = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])

# Make new empty list for indexing, with start and end
arrindex = []
arrindex.append([])
arrindex[-1].append(0)
arrindex[-1].append(0)
arrindex.append([])
arrindex[-1].append(len(imgsorted))
arrindex[-1].append(256)


In [ ]:
K = 4
# Fill array arrindex based on image data and K
for i in range(int(K/2)):
    length = len(arrindex)
    for i in range(length-1):
        # Get the partial list to work with
        partiallst = imgsorted[int(arrindex[i][0]):int(arrindex[i+1][0])]
        print("Partial list : " , partiallst)
        # Get index of last mean value in list. Return index+1 of the value
        where = np.where(imgsorted == int(partiallst.mean()))[0][-1]+1
        print("index of mean : " , where , " with value " , int(partiallst.mean()))
        # Insert index and value of mean to arrindex
        arrindex.append([])
        arrindex[-1].append(where)
        arrindex[-1].append(int(partiallst.mean()))
    arrindex.sort()


In [ ]:

for i in range (len(arrindex)-1):
    partiallst = imgsorted[int(arrindex[i][0]):int(arrindex[i+1][0])]
    arrindex[i+1].append(int(partiallst.mean()))

In [ ]:
arrindex.sort()
print(arrindex, len(arrindex))

In [ ]:
# Create copy of imgbnw
imgbnwquantized = imgbnw.copy()

# Structure:
# 
# array.shape[rows][column]
# 
#   -----> column
#  |
#  |
#  v
# rows
rows = imgbnwquantized.shape[0]
cols = imgbnwquantized.shape[1]

for i in range (rows):
    for j in range (cols):
        # imgbnwquantized[i][j] = 125
        for k in range (len(arrindex)-1):
            if(imgbnwquantized[i][j] > arrindex[k][1]) and (imgbnwquantized[i][j] <= arrindex[k+1][1]):
                imgbnwquantized[i][j] = arrindex[k+1][2]
                break
            else:
                continue 


In [ ]:
print(len(arrindex))
print(arrindex[0][1])
print(imgbnwquantized)

In [ ]:
cv.imshow("Nagyung Quantized",imgbnwquantized)
cv.waitKey(0)

In [ ]:
# Histogram for bnw image
plt.figure(figsize=(10,5))
plt.title('Quantized Histogram, K=4')
plt.hist(imgbnwquantized.ravel(), bins=256, range=(0, 256), color='black', alpha=0.4)
plt.show()